In [12]:
import pandas as pd
import feather 

# TODO:
# - patient/samplenrs as index (add for blood bank)
# - filter null SLE
# - "all" name

# Load data

In [13]:
other_imid = pd.read_spss("../data/interim/OtherIMID.sav") 
all = pd.read_spss("../data/interim/Alles 1e sample.sav") # SLE patients (also has non- and other-IMID data)
non_imid = pd.read_spss("../data/interim/Non-Imid control set.sav")
tmo = pd.read_csv("../data/interim/TMO.csv") # blood bank controls (also has data from SLE patients)

# load translation table of columns in TMO.csv vs. the .sav files
df_cols = pd.read_csv("../references/chip_colnames.csv", sep=";")

# Process

The `all` and `tmo` datasets contain data from more than one group

In [14]:
sle = all[all.SLE == 1] # keep only SLE patients
blood_bank = tmo[tmo.Class == "nonSLE"] # keep only blood bank controls

The chip columns are called differently in the `blood_bank` dataset than in the others.  `df_cols` keeps track of this:

In [15]:
df_cols

,TF,TB_all,TB_selection
0,Actinin,Actinin,Actinin
1,anti-IgE,antiIgE,NaN
2,ASCA,ASCA,ASCA
3,Beta2GP1,Beta2GP1,Beta2GP1
4,C1q,C1q,C1q
...,...,...,...
96,NaN,Strep15,NaN
97,NaN,Strep16,NaN
98,TIF1gamma,TIF1gamma,TIF1gamma
99,TPO,TPO,TPO


- `TF` are the names in `blood_bank`
- `TB_all` are the names in the other dfs
- `TB_selection` are names of the variables that should be most interesting (e.g. excluding control spots on the chip).

Each row corresponds to the same variable, but it might have a different name in each column!

Rename the columns in `blood_bank` as in the other data sets:

In [16]:
new_colnames = df_cols.TB_all[df_cols.TF.notnull()].tolist() # list of new names for blood bank columns
blood_bank = blood_bank.drop(columns='Class') # this column is in blood_bank, but not in the list (we'll add it back later)
blood_bank.columns = new_colnames # rename columns as in other datasets

We want only the rows that have an entry in all three columns: these are the variables we want to use

In [17]:
keep_cols = df_cols.dropna().TB_all.tolist() # names of variables that exist in both datasets, and that are of interest
keep_cols

['Actinin',
 'ASCA',
 'Beta2GP1',
 'C1q',
 'C3b',
 'Cardiolipin',
 'CCP1arg',
 'CCP1cit',
 'CENP',
 'CMV',
 'CollagenII',
 'CpGmot',
 'CRP1',
 'DFS70',
 'dsDNA2',
 'Enolasearg',
 'Enolasecit',
 'EphB2',
 'FcER',
 'Fibrillarin',
 'Ficolin',
 'GAPDH',
 'GBM',
 'H2Bp',
 'H2Bpac',
 'H4p',
 'H4pac',
 'Histones',
 'IFNLambda',
 'IFNOmega',
 'Jo1',
 'Ku',
 'LaSSB',
 'MBL2',
 'Mi2',
 'Nucleosome',
 'PCNA',
 'Pentraxin3',
 'PmScl100',
 'RA33',
 'RipP0',
 'RipP0peptide',
 'RipP1',
 'RipP2',
 'RNAPolIII',
 'RNP70',
 'RNPA',
 'RNPC',
 'Ro52',
 'Ro60',
 'RPP25ThTo',
 'Scl70',
 'SmBB',
 'SMP',
 'TIF1gamma',
 'TPO',
 'tTG']

In all datasets, keep only columns of interest

In [18]:
blood_bank = blood_bank.loc[:,keep_cols] # keep only columns of interest
other_imid = other_imid.loc[:,keep_cols] 
non_imid = non_imid.loc[:,keep_cols]
sle = sle.loc[:,keep_cols]

And row-bind all the data frames together

In [19]:
# add class to distinguish from others
blood_bank['Class'] = "nonSLE"
other_imid['Class'] = "IMID" 
non_imid['Class'] = "nonIMID" 
sle['Class'] = "SLE"
# join all data frames together by binding rows
df_all = pd.concat([sle, other_imid, non_imid, blood_bank])

In [20]:
df_all

,Actinin,ASCA,Beta2GP1,C1q,C3b,Cardiolipin,CCP1arg,CCP1cit,CENP,CMV,...,Ro52,Ro60,RPP25ThTo,Scl70,SmBB,SMP,TIF1gamma,TPO,tTG,Class
2,94.8911,1117.530,1328.0800,88.0225,115.4250,55.1872,42.1010,40.3490,65.5990,1177.9000,...,1200.420,3849.4900,36.8393,109.4460,222.8190,157.0750,283.8950,1011.080,170.611,SLE
4,99.9188,1295.260,119.1230,133.0480,59.4884,39.9630,39.0714,39.0714,35.5006,1023.6600,...,182.428,538.5860,47.0820,126.9600,96.4191,114.7650,84.1488,1111.850,146.075,SLE
5,121.3530,2636.220,38.4903,85.9066,117.8180,38.4903,42.0952,40.2934,53.7753,76.1163,...,407.374,3314.8400,51.9816,115.1650,721.0810,113.3960,154.8510,109.857,128.418,SLE
8,145.0990,995.634,509.1220,171.8770,179.0070,60.6069,67.8459,52.4473,203.9280,8717.1000,...,265.153,243.8840,66.9417,591.4510,291.7020,148.6730,4777.2800,765.190,211.928,SLE
9,66.0117,994.225,40.8654,184.3840,85.9921,44.1397,42.5038,43.3220,33.4571,3849.9300,...,1216.400,11671.5000,42.5038,76.4287,82.8095,66.8153,103.4140,716.172,237.993,SLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861,232.6780,2684.170,151.5480,612.2970,297.4040,116.1780,123.0410,127.9480,62.5797,384.4660,...,719.785,908.1220,130.8930,379.4510,171.2710,150.5630,171.2710,9299.620,1435.100,nonSLE
862,103.5940,968.906,314.0100,118.3070,87.9471,103.5940,72.3563,63.6152,47.1728,315.0110,...,142.908,113.3980,69.4401,902.3360,112.4170,85.0192,135.0250,13033.700,261.015,nonSLE
863,114.5400,1390.640,145.1040,197.6590,108.6430,99.8093,81.2161,79.2637,70.4906,672.3380,...,174.811,136.2150,297.5760,171.8350,126.3530,93.9293,61.7400,4645.730,340.774,nonSLE
864,161.1920,657.620,92.0882,283.0850,133.4540,119.6300,119.6300,120.6160,64.7232,139.3880,...,171.124,124.5630,113.7160,221.9570,327.3590,140.3780,90.1271,280.072,364.689,nonSLE


In [21]:
df_all['Class'].value_counts()

SLE        484
nonSLE     361
IMID       346
nonIMID    218
Name: Class, dtype: int64

# Write data

In [22]:
feather.write_dataframe(df_all, "../data/processed/imid.feather")